In [2]:
%gui qt
%matplotlib qt
import xmcd_projection
from xmcd_projection import deep_reload
deep_reload(xmcd_projection)
from xmcd_projection import *
from glob import glob
import os
from copy import copy, deepcopy
from skimage.io import imread, imsave

### Get file paths

In [4]:
folder_path = r"D:\Magnumfe\al6-1-1_peem_profile2_10mT15deg"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
msh_file = glob(os.path.join(data_folder, "*.msh"))[0]

# sort mag files
# indx = [int(mf.split('.')[-2]) for mf in mag_files]
indx = [int(mf.split('_')[-1].split('.')[0]) for mf in mag_files]
mag_files = [mf for _, mf in sorted(zip(indx, mag_files))]
# get the mesh and magnetisation
mesh0 = get_mesh(msh_file)
points0 = mesh0.points
mesh = ground_mesh(deepcopy(mesh0))
magnetisation, _ = get_magnumfe_magnetisation(mag_files[0])

In [5]:
struct = get_struct_from_mesh(mesh)
points, faces, tetra = get_mesh_data(mesh)
struct.faces = frozen_to_numpy(get_edge_faces(tetra))

### Make sure that the projection vector is correct and that the structure is oriented well

In [6]:
# need to mirror the structure
# struct.vertices[:, 0] *= -1
# p = get_projection_vector(-90, 15)
p = get_projection_vector(90, 16)
projected_structure = project_structure_byvector(struct, p=p)

In [7]:
vis = MagnumfeVisualizer(struct, projected_structure)
vis.show()

### Get the projection and piercing information and save immediately. Load if exists

In [8]:

deep_reload(xmcd_projection)
from xmcd_projection import *
t0 = time.time()
piercings_file = os.path.join(data_folder, "piercings.npy")
if os.path.exists(piercings_file):
    struct, struct_projected, piercings_list, p, mesh = load_piercing_data(piercings_file)
else:
    struct_projected, piercings_list, p = get_projection_with_piercings(mesh, p=p)
    save_piercing_data(piercings_file, struct, struct_projected, piercings_list, p, mesh)
        
print(time.time() - t0)

46673.059995889664


### Get the xmcd

#### Get the magnetisation from csv files
Note: sometimes if the mesh file has multiple parts, the paraview export and the original mesh coordinates are not in the same order. I add a function to fix that when necessary

In [9]:
magnetisation, coords = get_magnumfe_magnetisation(mag_files[-1])
# magnetisation *= -1
shuffle_file = os.path.join(data_folder, "shuffle_indx.npy")
try:
    shuffle_indx = np.load(shuffle_file)
except FileNotFoundError:
    print('File not found. Generating shuffle indx')
    shuffle_indx = get_shuffle_indx(coords, points0)
    np.save(shuffle_file, shuffle_indx)
# shuffle_indx = get_shuffle_indx(coords, points0)
magnetisation = magnetisation[shuffle_indx, :]

File not found. Generating shuffle indx


#### Get the colours and XMCD values

In [18]:
deep_reload(xmcd_projection)
from xmcd_projection import *
xmcd_value = get_xmcd_from_piercings_list(tetra, magnetisation, piercings_list)
mag_colors = get_struct_face_mag_color(struct, magnetisation)

In [24]:
vis.background_color

1

In [22]:
deep_reload(xmcd_projection)
from xmcd_projection import *
center_struct = [-100, 100, 0]
dist_struct = 1e5
center_peem = [600, -5000, 0]
dist_peem = 7e5

vis = MagnumfeVisualizer(struct, struct_projected, projected_xmcd=xmcd_value, struct_colors=mag_colors)
vis.update_view()
vis.show()
vis.set_camera(azi=18, center=center_peem, dist=dist_peem)

In [15]:
vis.update_colors(xmcd_value, mag_colors)
vis.set_camera(azi=18, center=center_peem, dist=dist_peem)

In [33]:

vis.struct_colors = get_struct_face_mag_color(struct, magnetisation, cmap_name='seismic')
vis.update_view()

In [23]:

vis.set_camera(azi=18, center=center_struct, dist=dist_struct)
vis.hide_projection()

#### Saving one render

In [17]:
vis.struct_colors

array([[0.71372549, 0.71372549, 0.71372549, 1.        ],
       [0.8627451 , 0.8627451 , 0.8627451 , 1.        ],
       [0.27058824, 0.27058824, 0.27058824, 1.        ],
       ...,
       [0.29019608, 0.29019608, 0.29019608, 1.        ],
       [0.32156863, 0.32156863, 0.32156863, 1.        ],
       [0.88235294, 0.88235294, 0.88235294, 1.        ]])

In [16]:
vis.save_render(os.path.join(folder_path, 'processed', 'shadow_final.png'))
blurred = vis.get_blurred_image(desired_background=0.7)
imsave(os.path.join(folder_path, 'processed', 'shadow_final_blurred.png'), (blurred*255).astype(np.uint8), check_contrast=False)

vis.set_camera(azi=18, center=center_struct, dist=dist_struct)
vis.hide_projection()
vis.save_render(os.path.join(folder_path, 'processed', 'structure_view.png'))

### Run for all magnetisations and save images

In [205]:
desired_background = 0.7 # if not None, adjusts the contrast so that the background is of the desired value in the blurred image
savedir = os.path.join(folder_path, 'processed', 'xmcd_projection')
savedir_blurred = savedir+'_blurred1'
savedir_structure = os.path.join(folder_path, 'processed', 'projected_structure')
for sd in (savedir, savedir_blurred, savedir_structure):
    if not os.path.exists(sd):
        os.makedirs(sd)

for mf in mag_files:
#     file_num = int(mf.split('.')[-2])
    file_num = int(mf.split('_')[-1].split('.')[0])
    # get the magnetisation and xmcd
    magnetisation, _ = get_magnumfe_magnetisation(mf)
    magnetisation = magnetisation[shuffle_indx, :]
    xmcd_value = get_xmcd_from_piercings_list(tetra, magnetisation, piercings_list)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    # update the view
    vis.update_colors(xmcd_value, mag_colors)
#     vis.set_camera(azi=18, center=center_peem, dist=dist_peem)
    file_name = 'step_{}.png'.format(file_num)
#     image = vis.save_render(os.path.join(savedir, file_name))
    # save the blurred image
    blurred = vis.get_blurred_image(desired_background=desired_background)
    imsave(os.path.join(savedir_blurred, file_name), (blurred*255).astype(np.uint8), check_contrast=False)
    # save the image of the structure
#     vis.set_camera(azi=18, center=center_struct, dist=dist_struct)
#     vis.hide_projection()
#     image = vis.save_render(os.path.join(savedir_structure, file_name))

In [192]:
folder_path

'D:\\Magnumfe\\al6-1-1_peem_profile1_001'

In [207]:
desired_background = 0.7 # if not None, adjusts the contrast so that the background is of the desired value in the blurred image

savedir = os.path.join(folder_path, 'processed', 'xmcd_projection_inv')
savedir_blurred = savedir+'_blurred1'
savedir_structure = os.path.join(folder_path, 'processed', 'projected_structure_inv')
for sd in (savedir, savedir_blurred, savedir_structure):
    if not os.path.exists(sd):
        os.makedirs(sd)
        
for mf in mag_files:
#     file_num = int(mf.split('.')[-2])
    file_num = int(mf.split('_')[-1].split('.')[0])
    magnetisation, _ = get_magnumfe_magnetisation(mf)
    magnetisation *= -1
    magnetisation = magnetisation[shuffle_indx, :]
    xmcd_value = get_xmcd_from_piercings_list(tetra, magnetisation, piercings_list, p=p)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    # update the view
    vis.update_colors(xmcd_value, mag_colors)
    vis.set_camera(azi=18, center=center_peem, dist=dist_peem)
    file_name = 'step_{}.png'.format(file_num)
#     image = vis.save_render(os.path.join(savedir, file_name))
    # save the blurred image
    blurred = vis.get_blurred_image(desired_background=desired_background)
    imsave(os.path.join(savedir_blurred, file_name), (blurred*255).astype(np.uint8), check_contrast=False)
#     # save the image of the structure
#     vis.set_camera(azi=18, center=center_struct, dist=dist_struct)
#     vis.hide_projection()
#     image = vis.save_render(os.path.join(savedir_structure, file_name))


### Import the saved images and make a movie

In [18]:
FPS = 10

out_file = os.path.join(folder_path, 'processed', 'projection_movie_inv.mp4')
image_files = glob(os.path.join(savedir, '*.png'))

indx = [int(imgf.split('.')[-2].split('_')[-1]) for imgf in image_files]
image_files = [imgf for _, imgf in sorted(zip(indx, image_files))]

# Define the codec and create VideoWriter object
fourcc =cv2.VideoWriter_fourcc('M','J','P','G') # Be sure to use lower case
img = cv2.imread(image_files[0])
out_writer = cv2.VideoWriter(out_file, fourcc, FPS, (img.shape[1], img.shape[0]))

i = 0
for imf in tqdm(image_files):
    img = cv2.imread(imf)
    # append to the video writer
    out_writer.write(img)
    
# Release everything if job is finished
out_writer.release()
cv2.destroyAllWindows()

print("The output video is {}".format(out_file))

100%|████████████████████████████████████████████████████████████████████████████████| 121/121 [00:02<00:00, 52.70it/s]


The output video is D:\Magnumfe\al6-1-1_peem_profile1\processed\projection_movie_inv.mp4


### Adding blur

In [90]:
img1

array([[0.25490196, 0.25490196, 0.25490196, ..., 0.25490196, 0.25490196,
        0.25490196],
       [0.25490196, 0.25490196, 0.25490196, ..., 0.25490196, 0.25490196,
        0.25490196],
       [0.25490196, 0.25490196, 0.25490196, ..., 0.25490196, 0.25490196,
        0.25490196],
       ...,
       [0.25490196, 0.25490196, 0.25490196, ..., 0.25490196, 0.25490196,
        0.25490196],
       [0.25490196, 0.25490196, 0.25490196, ..., 0.25490196, 0.25490196,
        0.25490196],
       [0.25490196, 0.25490196, 0.25490196, ..., 0.25490196, 0.25490196,
        0.25490196]])

In [94]:
plt.imshow((img1*255).astype(np.uint8))

In [120]:
savedir

'D:\\Magnumfe\\al6-1-1_peem_profile1_001\\processed\\xmcd_projection_blur'

In [122]:
from skimage.color import gray2rgb
from tqdm import tqdm_notebook as tqdm
folder_path1 = r"D:\Magnumfe\angles_initialisation"
projection_folder = os.path.join(folder_path1, 'processed', 'xmcd_projection_inv')
savedir = projection_folder + '_blur'
if not os.path.exists(savedir):
    os.makedirs(savedir)

image_files = glob(os.path.join(projection_folder, '*.png'))
for img_file in tqdm(image_files):
    img = imread(img_file)
    img1 = (gaussian(rgb2gray(rgba2rgb(img)), sigma=4)*255).astype(np.uint8)
    imsave(os.path.join(savedir, os.path.basename(img_file)), img1, check_contrast=False)
    

  0%|          | 0/14 [00:00<?, ?it/s]

In [84]:
plt.imshow(img1, cmap='gray')

# Angles study

In [65]:
folder_path = r"D:\Magnumfe\angles_initialisation"

data_folder = os.path.join(folder_path, "data")
mag_files = glob(os.path.join(data_folder, "*.csv"))
msh_file = glob(os.path.join(data_folder, "*.msh"))[0]

mesh = get_mesh(msh_file)
struct = get_struct_from_mesh(mesh)
p = get_projection_vector(90, 16)
projected_structure = project_structure_byvector(struct, p=p)

magnetisation, _ = get_magnumfe_magnetisation(mag_files[-1])
xmcd_value = get_xmcd_from_piercings_list(mesh, magnetisation, piercings_list)
mag_colors = get_struct_face_mag_color(struct, magnetisation)

In [66]:
vis = MagnumfeVisualizer(struct, struct_projected, xmcd_value, struct_colors=mag_colors)
vis.show()
vis.set_camera(azi=18, center=vis.get_structs_center(), dist=6e5)

In [60]:
os.path.basename(mag_files[0]).split('.')[0]

'al6-1-1_peem_angle-15'

In [ ]:
savedir = os.path.join(folder_path, 'processed', 'xmcd_projection')
if not os.path.exists(savedir):
    os.makedirs(savedir)
    
for mf in tqdm(mag_files):
    save_name = os.path.basename(mf).split('.')[0]+'.png'
    save_path = os.path.join(savedir, save_name)
    if os.path.exists(save_path):
        print('continuing')
        continue
    magnetisation, _ = get_magnumfe_magnetisation(mf)
#     magnetisation *= -1
    xmcd_value = get_xmcd_from_piercings_list(mesh, magnetisation, piercings_list)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    vis.update_colors(xmcd_value, mag_colors)
    image = vis.save_render(save_path)

In [70]:
savedir = os.path.join(folder_path, 'processed', 'xmcd_projection_inv')
if not os.path.exists(savedir):
    os.makedirs(savedir)
    
for mf in tqdm(mag_files):
    save_name = os.path.basename(mf).split('.')[0]+'.png'
    save_path = os.path.join(savedir, save_name)
    if os.path.exists(save_path):
        print('continuing')
        continue
    magnetisation, _ = get_magnumfe_magnetisation(mf)
    magnetisation *= -1
    xmcd_value = get_xmcd_from_piercings_list(mesh, magnetisation, piercings_list)
    mag_colors = get_struct_face_mag_color(struct, magnetisation)
    vis.update_colors(xmcd_value, mag_colors)
    image = vis.save_render(save_path)



  0%|                                                                                           | 0/14 [00:00<?, ?it/s]

continuing




  7%|█████▉                                                                             | 1/14 [00:00<00:01,  8.20it/s]

continuing
continuing
continuing
continuing




 43%|███████████████████████████████████▌                                               | 6/14 [00:13<00:07,  1.11it/s]

continuing
continuing




 64%|█████████████████████████████████████████████████████▎                             | 9/14 [00:27<00:09,  1.98s/it]

continuing
continuing
continuing
continuing
continuing




100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:27<00:00,  1.95s/it]

In [71]:
folder_path

'D:\\Magnumfe\\angles_initialisation'